<a href="https://colab.research.google.com/github/yejin-park/MachineLearning/blob/main/%EB%B2%84%EC%8A%A4%EC%9A%B4%ED%96%89%EC%8B%9C%EA%B0%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### train['next_arrive_time'].min() 인 route_id를 출력하는 방법?

#버스 운행시간 데이터를 활용할 데이터 분석

*   버스가 다음 정류장에 도착하기 위해서 걸릴 시간을 예측



#1. Library & Data

> folium : 지리정보 시각화 패키지



In [3]:
import pandas as pd #판다스 패키지 불러오기
import lightgbm as lgb
import folium  # 지리정보 시각화 패키지

In [4]:
#데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/bus/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/bus/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/bus/submission.csv')

#2. EDA

In [5]:
train.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude,next_arrive_time
0,0,2019-10-15,405136001,7997025,360-1,33.456267,126.551750,제주대학교입구,06시,266.0,제대마을,33.457724,126.554014,24
1,1,2019-10-15,405136001,7997025,360-1,33.457724,126.554014,제대마을,06시,333.0,제대아파트,33.458783,126.557353,36
2,2,2019-10-15,405136001,7997025,360-1,33.458783,126.557353,제대아파트,06시,415.0,제주대학교,33.459893,126.561624,40
3,3,2019-10-15,405136001,7997025,360-1,33.479705,126.543811,남국원(아라방면),06시,578.0,제주여자중고등학교(아라방면),33.484860,126.542928,42
4,4,2019-10-15,405136001,7997025,360-1,33.485662,126.494923,도호동,07시,374.0,은남동,33.485822,126.490897,64



*   route_id : 버스 노선 ID
*   vh_id : 버스 ID
*   route_nm : 버스 노선 실제 번호
*   now_latitude / now_longitude : 현재 정류소의 위도 / 경도
*   now_station : 현재 정류소 이름
*   now_arrive_time : 현재 정류장에 도착한 시간

*   distance :  현재 정류장에서 다음 정류장까지 실제 이동한 거리
*   next_station / latitude / longitude : 다음정류장의 ~
*   next_arrive_time : [예측할 값] 다음 정류장에 도착할 때 까지 걸린 시간(단위:초)



In [6]:
test.head()

,id,date,route_id,vh_id,route_nm,now_latitude,now_longitude,now_station,now_arrive_time,distance,next_station,next_latitude,next_longitude
0,210457,2019-10-29,405136001,7997025,360-1,33.457724,126.554014,제대마을,07시,333.0,제대아파트,33.458783,126.557353
1,210458,2019-10-29,405136001,7997025,360-1,33.458783,126.557353,제대아파트,07시,415.0,제주대학교,33.459893,126.561624
2,210459,2019-10-29,405136001,7997025,360-1,33.478867,126.483833,한라중학교/부영아파트,08시,417.0,대림2차아파트,33.478850,126.488350
3,210460,2019-10-29,405136001,7997025,360-1,33.478850,126.488350,대림2차아파트,08시,334.0,연동대림1차아파트,33.480700,126.489933
4,210461,2019-10-29,405136001,7997025,360-1,33.480700,126.489933,연동대림1차아파트,08시,550.0,케이티앤지,33.482077,126.485355


In [7]:
submission.head()

,id,next_arrive_time
0,210457,0
1,210458,0
2,210459,0
3,210460,0
4,210461,0


In [8]:
train.date.min()

'2019-10-15'

In [9]:
train['date'].min()

'2019-10-15'

In [10]:
print(train['date'].min())

2019-10-15


In [11]:
print("train MIN ", train['date'].min())
print("train MAX ", train['date'].max())

print("test MIN ", test['date'].min())
print("test MAX ", test['date'].max())

train MIN  2019-10-15
train MAX  2019-10-28
test MIN  2019-10-29
test MAX  2019-11-05


train 데이터 -> test 데이터 기간 순

In [12]:
print(train.shape)
print(test.shape)
print(submission.shape)

(210457, 14)
(91774, 13)
(91774, 2)


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210457 entries, 0 to 210456
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                210457 non-null  int64  
 1   date              210457 non-null  object 
 2   route_id          210457 non-null  int64  
 3   vh_id             210457 non-null  int64  
 4   route_nm          210457 non-null  object 
 5   now_latitude      210457 non-null  float64
 6   now_longitude     210457 non-null  float64
 7   now_station       210457 non-null  object 
 8   now_arrive_time   210457 non-null  object 
 9   distance          210457 non-null  float64
 10  next_station      210457 non-null  object 
 11  next_latitude     210457 non-null  float64
 12  next_longitude    210457 non-null  float64
 13  next_arrive_time  210457 non-null  int64  
dtypes: float64(5), int64(4), object(5)
memory usage: 22.5+ MB


In [14]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91774 entries, 0 to 91773
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               91774 non-null  int64  
 1   date             91774 non-null  object 
 2   route_id         91774 non-null  int64  
 3   vh_id            91774 non-null  int64  
 4   route_nm         91774 non-null  object 
 5   now_latitude     91774 non-null  float64
 6   now_longitude    91774 non-null  float64
 7   now_station      91774 non-null  object 
 8   now_arrive_time  91774 non-null  object 
 9   distance         91774 non-null  float64
 10  next_station     91774 non-null  object 
 11  next_latitude    91774 non-null  float64
 12  next_longitude   91774 non-null  float64
dtypes: float64(5), int64(3), object(5)
memory usage: 9.1+ MB


!!! train & test 데이터 _ 결측값이 존재하지 않음 !!!

In [15]:
train.describe()

,id,route_id,vh_id,now_latitude,now_longitude,distance,next_latitude,next_longitude,next_arrive_time
count,210457.000000,2.104570e+05,2.104570e+05,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000,210457.000000
mean,105228.000000,4.052491e+08,7.988694e+06,33.434528,126.603451,490.256100,33.434711,126.603687,85.380824
std,60753.847139,9.132404e+04,6.774077e+03,0.102350,0.123961,520.563932,0.102224,0.123838,85.051170
min,0.000000,4.051360e+08,7.983000e+06,33.244382,126.473300,97.000000,33.244382,126.473300,6.000000
25%,52614.000000,4.051365e+08,7.983093e+06,33.325283,126.523900,291.000000,33.325283,126.524550,44.000000
50%,105228.000000,4.053201e+08,7.983431e+06,33.484667,126.551050,384.000000,33.484860,126.551050,66.000000
75%,157842.000000,4.053201e+08,7.997041e+06,33.500197,126.650322,542.000000,33.500228,126.650322,102.000000
max,210456.000000,4.053281e+08,7.997124e+06,33.556167,126.935188,7461.000000,33.556167,126.935188,2996.000000


In [16]:
### 고유값 확인
len(train['now_station'].unique())

348

In [17]:
### drop_duplicates() 중복값 제거
len(train['now_station'].drop_duplicates())

348

In [18]:
train[['route_id', 'vh_id']]

,route_id,vh_id
0,405136001,7997025
1,405136001,7997025
2,405136001,7997025
3,405136001,7997025
4,405136001,7997025
...,...,...
210452,405328102,7983486
210453,405328102,7983486
210454,405328102,7983486
210455,405328102,7983486


~ 210456행

In [19]:
train[['route_id', 'vh_id']].drop_duplicates()

,route_id,vh_id
0,405136001,7997025
181,405136001,7997034
381,405136001,7997035
539,405136001,7997040
717,405136001,7997044
...,...,...
20226,405136522,7997004
26037,405320116,7983225
27098,405320122,7983067
45506,405328102,7983401


~ 100467행

In [20]:
### route_id(노선)별로 vh_id(운행되고있는 차량) 수
train[['route_id', 'vh_id']].drop_duplicates().groupby('route_id').count()

,vh_id
route_id,
405136001,10
405136002,5
405136007,1
405136012,3
405136521,6
405136522,15
405320111,3
405320112,5
405320113,1


In [21]:
### 노선 별로 다음 정류장에 도착하기까지 걸리는 시간
train[['route_id', 'next_arrive_time']].groupby('route_id').mean()
##### 매우 상이한 것을 알 수 있음

,next_arrive_time
route_id,
405136001,92.063688
405136002,108.270852
405136007,94.172947
405136012,111.322633
405136521,114.673939
405136522,101.339188
405320111,68.901710
405320112,67.602392
405320113,68.403649


In [47]:
train.isna().sum()

id                  0
date                0
route_id            0
vh_id               0
route_nm            0
now_latitude        0
now_longitude       0
now_station         0
now_arrive_time     0
distance            0
next_station        0
next_latitude       0
next_longitude      0
next_arrive_time    0
dtype: int64

결측치 없음

##folium.Map()

*   location 옵션에 있는 좌표를 중심으로 지도 시각화



In [22]:
coord = [35, 129]
map_osm = folium.Map(location = coord)
map_osm

In [23]:
### 어느정도 확대해서 볼 것인가 ###
map_osm = folium.Map(location=coord, zoom_start=10) 
map_osm

In [24]:
### 지도유형 _ tiles = 어떤 스타일의 지도 ###
map_osm = folium.Map(location = coord, tiles = 'Stamen Terrain')
map_osm

In [25]:
map_osm = folium.Map(location = coord, tiles = 'Stamen Toner')
map_osm

In [26]:
coord = [35.2335123, 129.0810047]

map_osm = folium.Map(location = coord, zoom_start =12)

#,Marker 마커 생성 뒤 지도에 추가해줌
folium.Marker(coord, popup='부산대학교', tooltip = '부산대학교 tooltip').add_to(map_osm)

#save 한국어 호환 원활하게
map_osm.save('index.html')
map_osm


In [27]:
map_osm = folium.Map(location=coord, zoom_start=12)
folium.Marker(coord, popup='부산대학교', icon=folium.Icon(icon='home')).add_to(map_osm)
map_osm

------------------------------


In [28]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)

folium.Marker([37.566345, 126.977893], popup='서울특별시청', icon=folium.Icon(color='red', icon='info-sign')).add_to(map_osm)

##Circle##
folium.Circle([37.4600, 126.9519], tooltip='서울대학교', radius=3000, color="green").add_to(map_osm)
folium.CircleMarker([37.5662, 126.9386], radius=20, poopup='연세대학교', color="blue", fill_color="crimson").add_to(map_osm)
map_osm

In [29]:
map_osm = folium.Map(location=[37.541, 126.986], zoom_start=11)

### number_of_sides / fill_opacity ###
folium.RegularPolygonMarker([37.566345, 126.977893], popup='서울특별시청',fill_color='red', color='red', number_of_sides=3, radius=30, fill_opacity=0.2 ).add_to(map_osm)
folium.RegularPolygonMarker([37.4600, 126.9519], popup='서울대학교', fill_color='green', color='green', number_of_sides=4, radius=10, fill_opacity=0.4).add_to(map_osm)
folium.RegularPolygonMarker([37.5662, 126.9386], popup='연세대학교', fill_color='blue', color='blue', number_of_sides=6).add_to(map_osm)
map_osm

------------------------------------

In [30]:
##########################################################
### 위 내용을 데이터 분석에 어떻게 적용시킬 수 있을 까 ###
##########################################################

# (노선 별로 다음 정류장에 도착하기까지 걸리는 시간)
train[['route_id', 'next_arrive_time']].groupby('route_id').mean()


,next_arrive_time
route_id,
405136001,92.063688
405136002,108.270852
405136007,94.172947
405136012,111.322633
405136521,114.673939
405136522,101.339188
405320111,68.901710
405320112,67.602392
405320113,68.403649


###가설

(노선 별로 다음 정류장에 도착하기까지 걸리는 시간) 이 적을 수록 교통 체증이 적은 외곽지역일 것이다


In [34]:
map_data = train[['route_id', 'now_latitude', 'now_longitude', 'now_station']]

###route_id :405136521 / next_arrive_time :114.673939 _ 가장 오래걸리는 경로
map_bus_route = map_data[map_data['route_id'] == 405136521].drop_duplicates("now_station")

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10.5)

for item in map_bus_route.index:
  lat = map_bus_route.loc[item, 'now_latitude']
  long = map_bus_route.loc[item, 'now_longitude']

  folium.Marker([lat, long], popup = map_bus_route.loc[item, 'now_station'],
                icon = folium.Icon(color = 'red', icon = 'info-sign')).add_to(map_osm)

map_osm.save('index.html')

map_osm

In [40]:
train['next_arrive_time'].min()

6


In [46]:
### 다음 정류장에 도착하기까지 걸리는 시간이 가장 작은 경로

map_data = train[['route_id', 'now_latitude', 'now_longitude', 'now_station']]
map_bus_route = map_data[map_data['route_id'] == 405320122	].drop_duplicates("now_station")

map_osm = folium.Map(location=[33.4134, 126.5190], zoom_start = 10)

for item in map_bus_route.index:
  lat = map_bus_route.loc[item, 'now_latitude']
  long = map_bus_route.loc[item, 'now_longitude']

  folium.Marker([lat, long], popup = map_bus_route.loc[item, 'now_station'],
                icon = folium.Icon(color = 'red', icon = 'info-sign')).add_to(map_osm)

map_osm.save('index.html')

map_osm